# XGboost model

In [8]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')


In [9]:
anomoly  = False
# load data either by anomoly or not
if anomoly:
    X=pd.read_csv("https://raw.githubusercontent.com/KelvinYQC/msia420PA_project/main/Data/with_anomaly.csv")
else:
    X=pd.read_csv("https://raw.githubusercontent.com/KelvinYQC/msia420PA_project/main/Data/without_anomaly.csv")
y = X['booking_status']
X.drop(['booking_status'], axis = 1, inplace = True)


In [10]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, test_size=None, train_size=0.3)

In [11]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[("scaler", StandardScaler()), ("clf", xgb.XGBClassifier())])


parameters = {
    'clf__learning_rate': [0.001, 0.01, 0.05, 0.1, None],
    'clf__max_depth': [2,5,7,10,15, 30, None]
}

clf = RandomizedSearchCV(estimator=pipeline,
                         param_distributions = parameters,
                         n_iter=20,
                         verbose=1,
                         cv = 5,
                         scoring = 'roc_auc')
clf_xgb = clf.fit(X_train, y_train)
print("Best parameters:", clf.best_params_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


KeyboardInterrupt: 

In [35]:
from sklearn.metrics import roc_auc_score

y_pred = clf_xgb.predict(X_test)
classifier_score_xgb = roc_auc_score(y_test, y_pred)

print("The resulting roc auc score: ")
print(round(classifier_score_xgb,3))

0.8616239058937933

The AUC from XGboost model is 0.8616.